In [13]:
import geopandas as gpd
from tobler.area_weighted.area_interpolate import _area_tables_binning
import numpy as np
import pandas as pd

In [2]:
clc = "/Users/martin/Downloads/u2018_clc2018_v2020_20u1_geoPackage/DATA/U2018_CLC2018_V2020_20u1.gpkg"

In [3]:
lsoa = gpd.read_file("/Users/martin/Downloads/Lower_Layer_Super_Output_Areas__December_2011__Boundaries_Full_Clipped__BFC__EW_V3-shp/Lower_Layer_Super_Output_Areas__December_2011__Boundaries_Full_Clipped__BFC__EW_V3.shp")

In [4]:
corine = gpd.read_file(clc, layer='U2018_CLC2018_V2020_20u1', bbox=lsoa.to_crs(3035))

/Users/martin/Git/geopandas/martinfleis/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [5]:
corine = corine.to_crs(lsoa.crs)

In [6]:
source_df = corine
target_df = lsoa
spatial_index = "auto"
categorical_variables = ['Code_18']

In [7]:
%time table = _area_tables_binning(source_df, target_df, spatial_index)

CPU times: user 1h 9min 26s, sys: 21.9 s, total: 1h 9min 48s
Wall time: 1h 10min 9s


In [11]:
%%time
categorical = {}
for variable in categorical_variables:
    unique = source_df[variable].unique()
    for value in unique:
        mask = source_df[variable] == value
        categorical[f"{variable}_{value}"] = np.asarray(table[mask].sum(axis=0))[0]

CPU times: user 8min 50s, sys: 2.97 s, total: 8min 53s
Wall time: 9min 2s


In [15]:
%%time
categorical = pd.DataFrame(categorical, index=target_df.index)
categorical = categorical.div(target_df.area, axis='rows')

CPU times: user 172 ms, sys: 24.1 ms, total: 196 ms
Wall time: 183 ms


In [16]:
result = pd.concat([target_df, categorical], axis=1)

In [17]:
result

,OBJECTID,LSOA11CD,LSOA11NM,BNG_E,BNG_N,LONG_,LAT,Shape_Leng,Shape__Are,Shape__Len,...,Code_18_122,Code_18_511,Code_18_512,Code_18_132,Code_18_333,Code_18_332,Code_18_523,Code_18_334,Code_18_222,Code_18_244
0,1,E01000001,City of London 001A,532123,181632,-0.097140,51.5182,2635.767993,1.298653e+05,2635.767993,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,E01000002,City of London 001B,532480,181715,-0.091970,51.5188,2707.881853,2.284196e+05,2707.881853,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,E01000003,City of London 001C,532239,182033,-0.095320,51.5217,1224.638192,5.905435e+04,1224.638192,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,E01000005,City of London 001E,533581,181283,-0.076270,51.5147,2275.805348,1.895777e+05,2275.805348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,E01000006,Barking and Dagenham 016A,544994,184274,0.089317,51.5387,1966.092607,1.465370e+05,1966.092607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34748,34749,W01001954,Cardiff 006F,312959,180574,-3.255820,51.5173,5567.813286,6.400342e+05,5567.813286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34749,34750,W01001955,Swansea 025F,265633,193182,-3.942370,51.6214,6037.602710,6.871190e+05,6037.602710,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34750,34751,W01001956,Swansea 023E,260586,192621,-4.015000,51.6151,13127.898240,2.500337e+06,13127.898240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34751,34752,W01001957,Swansea 025G,265337,192555,-3.946400,51.6157,2829.597133,2.139171e+05,2829.597133,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
